<a href="https://colab.research.google.com/github/kamalzada/Machine-Learning-Projects/blob/master/pd.read_html.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import string

In [122]:
#On a single url (each correspond to a fighter)
df = pd.read_html('http://ufcstats.com/fighter-details/1338e2c7480bdf9e')

In [123]:
#Getting the table format
df[0] 

,W/L,Fighter,Kd,Str,Td,Sub,Event,Method,Round,Time
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,win,Israel Adesanya Robert Whittaker,1 0,79 59,0 4,0 0,"UFC 271: Adesanya vs. Whittaker 2 Feb. 12, 2022",U-DEC,5.0,5:00
2,win,Israel Adesanya Marvin Vettori,0 0,96 58,0 4,0 0,"UFC 263: Adesanya vs. Vettori 2 Jun. 12, 2021",U-DEC,5.0,5:00
3,loss,Israel Adesanya Jan Blachowicz,0 0,78 107,0 3,0 0,"UFC 259: Blachowicz vs. Adesanya Mar. 06, 2021",U-DEC,5.0,5:00
4,win,Israel Adesanya Paulo Costa,1 0,55 12,0 0,0 0,"UFC 253: Adesanya vs. Costa Sep. 26, 2020",KO/TKO Punch,2.0,3:59
5,win,Israel Adesanya Yoel Romero,0 0,48 40,0 0,0 0,"UFC 248: Adesanya vs. Romero Mar. 07, 2020",U-DEC,5.0,5:00
6,win,Israel Adesanya Robert Whittaker,2 0,40 32,0 0,0 0,"UFC 243: Whittaker vs. Adesanya Oct. 05, 2019",KO/TKO Punch,2.0,3:33
7,win,Israel Adesanya Kelvin Gastelum,4 0,109 91,0 1,2 0,"UFC 236: Holloway vs. Poirier 2 Apr. 13, 2019",U-DEC,5.0,5:00
8,win,Israel Adesanya Anderson Silva,0 0,65 31,0 0,0 0,"UFC 234: Adesanya vs. Silva Feb. 09, 2019",U-DEC,3.0,5:00
9,win,Israel Adesanya Derek Brunson,3 0,13 3,0 0,0 0,"UFC 230: Cormier vs. Lewis Nov. 03, 2018",KO/TKO Punches,1.0,4:51


In [124]:
#Acquiring all the fighter links 
letters = string.ascii_lowercase
fighter_ids = {}

for letter in letters:
    pages = requests.get(f'http://ufcstats.com/statistics/fighters?char={letter}&page=all')
    soup = BeautifulSoup(pages.text, 'lxml')   
    fighter_ids[letter] = soup.find_all('a', attrs = {'class': "b-link b-link_style_black"})

In [125]:
fighter_links = []

for letter in letters:
    for fighter in fighter_ids[letter]:
        fighter_links.append(fighter['href'])
fighter_links = list(pd.Series(fighter_links).to_frame()[0].unique())
len(fighter_links)


3741

In [62]:
#Using requests and Pandas' read_html function together 
%%time
fighter_table = [[0 for f in range(0)] for f in range(len(fighter_links))]

for key, value in enumerate(fighter_links):
  page = requests.get(f'{value}')
  fighter_table[key] = pd.read_html(page.text)[0]

CPU times: user 46.6 s, sys: 2.49 s, total: 49.1 s
Wall time: 34min 38s


In [101]:
#Engineering the knockout ratio values by dividing the tko/ko combined with win length to the entire win length
knockout_ratio = []

for index in range(len(fighter_table)):
  try:
    knockout_ratio.append(len(fighter_table[index][(fighter_table[index]['W/L']=='win') & (fighter_table[index]['Method'].str.contains('KO/TKO'))])/len(fighter_table[index][fighter_table[index]['W/L']=='win']))
  except:
    knockout_ratio.append('Insufficient fights')  


In [127]:
#Switching to a one-dimensional list
tko_ratio = [f for f in knockout_ratio]
df = pd.concat(map(pd.read_csv, ['fighter_stance.csv']))

In [128]:
#Converting that list into a Series
df['knockout_ratio'] = pd.Series(tko_ratio).to_frame()

In [133]:
df.iloc[70:75,:] 

,name,last_name,stance,wins,losses,SLpM,fights,knockout_ratio
70,Thomas,Almeida,Southpaw,22,5,4.84,27,0.8
71,Ericka,Almeida,Orthodox,7,3,1.07,10,Insufficient fights
72,Estefani,Almeida,Orthodox,7,3,3.13,10,Insufficient fights
73,Jailton,Almeida,Orthodox,15,2,3.28,17,0.5
74,Lucas,Almeida,Orthodox,12,1,6.40,13,Insufficient fights


In [121]:
#Not included in the test
df[df['knockout_ratio']=='Insufficient fights'].shape[0]

1699